In [142]:
from tensorflow.keras import Sequential
from tensorflow.keras.datasets import reuters
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer

# 파일 불러오기

In [143]:
# 데이터 셋 posneg.xlsx
df = pd.read_csv('./도레미파솔 3차 크롤링/마지막 학습용 데이터/taste_df.csv', encoding='utf-16')

In [144]:
df.columns

Index(['Unnamed: 0', 'taste', 'customer_review'], dtype='object')

In [145]:
df.head(5)

,Unnamed: 0,taste,customer_review
0,14,0.0,"2인분 시켜서 반찬 좀 2명 분 달랬더니 매번 하나만 챙겨주네, 짜증"
1,52,0.0,커피도 반만 들어있고ㅠ음식도 성의없어서 이젠 주문안하고 싶네요ㅠ
2,54,0.0,된장찌개와 돼지구이 쌈밥을 시켰습니다 예상도착시간이 되도 아무 연락이 없어 가게에 ...
3,96,0.0,요청사항은 안보시나봐요..ㅡ 사이다 시켰는데 콜라로 오고. 주문사항이랑 요청사항 확...
4,97,0.0,김치찌개가 부대찌개도아니고 상한맛같앗어요 다른건 다 맛잇었는데 신뢰가 확..ㅜ 작년...


In [146]:
len(df)

313540

In [147]:
neg_test = df[df['taste']==0]
len(neg_test)

14361

In [148]:
pos_test = df[df['taste']==1]
len(pos_test)

299179

In [149]:
df = df.sample(frac=1).reset_index(drop=True)
df

,Unnamed: 0,taste,customer_review
0,29704,1.0,너무 맛있어요 저번에 시켰는데 또 먹으려구요~
1,30971,1.0,배달 2시간 다 되서 왔네여 어쨌든 잘 먹었습니다
2,179986,1.0,맛있었어요! 돈가스양이 밥보다 많은 거 같아요 파인애플은 쏘쏘 근데 왔을때 김치랑 ...
3,14306,1.0,양은 좀그런데 맛은 따봉입니다
4,198228,0.0,양 ㅡ좋고요 배달 좋고요 맛은 저한테는좀 안맏는것가터요 김치는 정말맛있고요 원래 족...
...,...,...,...
313535,317334,1.0,예상시간보다 25분 늦음 ㅜ
313536,114793,1.0,양도 푸짐하고 맛있게 잘 먹었어요
313537,50459,1.0,배달은 거의 한시간 넘게 걸려서 안좋긴 했지만 음식은 맛있었어요~ 초밥이랑 캘리포니...
313538,128437,1.0,항상 너무 맛있어용


In [150]:
X = df['customer_review']

In [151]:
y = df['taste']

In [152]:
X

0                                 너무 맛있어요 저번에 시켰는데 또 먹으려구요~
1                               배달 2시간 다 되서 왔네여 어쨌든 잘 먹었습니다
2         맛있었어요! 돈가스양이 밥보다 많은 거 같아요 파인애플은 쏘쏘 근데 왔을때 김치랑 ...
3                                          양은 좀그런데 맛은 따봉입니다
4         양 ㅡ좋고요 배달 좋고요 맛은 저한테는좀 안맏는것가터요 김치는 정말맛있고요 원래 족...
                                ...                        
313535                                      예상시간보다 25분 늦음 ㅜ
313536                                   양도 푸짐하고 맛있게 잘 먹었어요
313537    배달은 거의 한시간 넘게 걸려서 안좋긴 했지만 음식은 맛있었어요~ 초밥이랑 캘리포니...
313538                                           항상 너무 맛있어용
313539                                   언제먹어도 역시 맛있는 넘버원!!
Name: customer_review, Length: 313540, dtype: object

In [153]:
y

0         1.0
1         1.0
2         1.0
3         1.0
4         0.0
         ... 
313535    1.0
313536    1.0
313537    1.0
313538    1.0
313539    1.0
Name: taste, Length: 313540, dtype: float64

# 시작

In [154]:
#KoNLPy패키지에는 Hannanum, Kkma, Komoran, Mecab, Okt(Twitter)등의 클래스들이 있습니다
# https://mr-doosun.tistory.com/22
# Okt (Twitter) Class
# 해당 분석기는 오픈 소스 한국어 분석기이고, 과거 트위터 형태소 분석기였습니다.

# KoNLPy 문서에 따른 클래스간의 성능 비교
# https://konlpy.org/ko/v0.4.3/morph/

# morphs  텍스트에서 형태소를 반환한다 / 형태소 : 일정한 의미가 있는 가장 작은 말의 단위
# nouns   텍스트에서 명사를 반환한다
# phrases 텍스트에서 어절을 뽑아낸다
# pos     텍스트에서 품사 정보를 부착하여 반환한다

In [155]:
# okt(Open Korean Text) 
from konlpy.tag import Okt
okt = Okt()

In [156]:
# okt test
okt.morphs('KoNLPy패키지에는 Hannanum, Kkma, Komoran, Mecab, Okt(Twitter)등의 클래스들이 있습니다.')

['KoNLPy',
 '패키지',
 '에는',
 'Hannanum',
 ',',
 'Kkma',
 ',',
 'Komoran',
 ',',
 'Mecab',
 ',',
 'Okt',
 '(',
 'Twitter',
 ')',
 '등',
 '의',
 '클래스',
 '들',
 '이',
 '있습니다',
 '.']

In [157]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 84)

In [158]:
len(X_train),len(y_train), len(X_test), len(y_test)

(219478, 219478, 94062, 94062)

In [159]:
y_test

265365    1.0
167301    1.0
159644    1.0
11796     1.0
51440     1.0
         ... 
104340    1.0
256567    1.0
212675    1.0
24973     1.0
39974     1.0
Name: taste, Length: 94062, dtype: float64

# TF-IDF (+파라미터)

In [160]:
# min-df DF(document-frequency)의 최소 빈도값 <-> max-df
# n-gram 것은 단어의 묶음
# analyzer = 'word'라고 설정시, 학습의 단위를 단어로 (ex - home, go, my ...)
# analyzer = 'char'라고 설정시, 학습의 단위를 글자로 (ex - a, b, c, d ...)
# ngram_range = (1, 2)라고 한다면, 단어의 묶음을 1개부터 2개까지 설정
# go home', 'very good'과 같은 2개 짜리 묶음도 인덱스를 받게 되는 것
# 단어가 묶여야 비로소 의미를 가지는 것들을 위해
# max_feature는 tf-idf vector의 최대 feature를 설정 /  사전에 EDA를 통해서 데이터의 length를 확인


from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(tokenizer=okt.morphs, ngram_range=(1,2), min_df=3, max_df=0.9)
tfv.fit(X_train)
tfv_X_train = tfv.transform(X_train)
tfv_X_train

C:\Users\smhrd\.conda\envs\DeepLearning\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


<219478x147127 sparse matrix of type '<class 'numpy.float64'>'
	with 4911353 stored elements in Compressed Sparse Row format>

In [161]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

clf = LogisticRegression(random_state=0)
params = {'C': [1,3,5,7,9]}
grid_cv = GridSearchCV(clf, param_grid = params, cv=4, scoring='accuracy', verbose=1)
grid_cv.fit(tfv_X_train, y_train)

Fitting 4 folds for each of 5 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\smhrd\.conda\envs\DeepLearning\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\smhrd\.conda\envs\DeepLearning\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documenta

C:\Users\smhrd\.conda\envs\DeepLearning\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\smhrd\.conda\envs\DeepLearning\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/

GridSearchCV(cv=4, estimator=LogisticRegression(random_state=0),
             param_grid={'C': [1, 3, 5, 7, 9]}, scoring='accuracy', verbose=1)

In [162]:
grid_cv.best_params_

{'C': 9}

In [163]:
grid_cv.best_score_

0.9728947781461874

In [ ]:
tfv_X_test=tfv.transform(X_test)
grid_cv.best_estimator_.score(tfv_X_test,y_test)

# 테스트

In [ ]:
TestSet=pd.read_csv('review_TestSet.csv',encoding='utf-16')

In [ ]:
TestSet

In [ ]:
TestSet.iloc[1150:1155]['reviews']

In [ ]:
my_review = tfv.transform(TestSet['reviews'])
result=grid_cv.best_estimator_.predict(my_review)

In [ ]:
# 1긍정 2부정
result[1150]